# 从远程数据库中读取数据

In [1]:
import pymysql
import numpy as np
import pandas as pd
import re
import jieba
from collections import Counter
import os

In [2]:
def cut(string):
    return ' '.join(jieba.cut(string))

def token(string):
    string = re.findall('[\d|\w]+', string)
    return ' '.join(string)

def deal(string):
    string = token(string)
    return cut(string)

In [3]:
string = '大家好, 我在学习NLP'
deal(string)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.444 seconds.
Prefix dict has been built succesfully.


'大家 好   我 在 学习 NLP'

In [22]:
db = pymysql.connect("cdb-q1mnsxjb.gz.tencentcdb.com", "root", "A1@2019@me", "news_chinese", port=10102)
cursor = db.cursor()

In [23]:
sql = """select count(content) from sqlResult_1558435"""
try:
    cursor.execute(sql)
except:
    print("Error")
length_corpus = cursor.fetchall()

In [24]:
import time
start = time.time()
with open('news.txt', 'a') as f:
    for i in range(0,length_corpus[0][0], 10):
        sql = """select content from sqlResult_1558435 where id>={} and id < {}""".format(i, i+10)
        try:
            cursor.execute(sql)
        except:
            print("Error")

        news = cursor.fetchall()

        for j in range(len(news)):
            data = news[j][0]
            text = deal(data)
            f.write(text + '\n')
        break
print(time.time() - start)
cursor.close()
db.close()

5.5074779987335205


In [25]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
news_word2ve = Word2Vec(LineSentence('news.txt'), size = 35, workers=8)
#news_word2ve.most_similar('说')

# 维基百科和新闻语料库制作的词向量

In [133]:
path = '/home/zhaodao/下载/text/'
def get_data(path):
    dir_name = os.listdir(path)
    STRING = []
    for name in dir_name:
        print("name", name)
        one_path = os.path.join(path, name)
        all_ = os.listdir(one_path)
        for file in all_:
            print("file", file)
            data = open(os.path.join(one_path, file))
            yield data.readlines()

data = get_data(path)

In [ ]:
with open('news-sentences-xut.txt', 'a') as f:
    while True:
        try:
            b = next(data)
            text = [deal(string) for string in b if string!='\n']
            for n in text:
                f.write(n + '\n')
        except:
            break

file wiki_28
file wiki_33
file wiki_14
file wiki_88
file wiki_06
file wiki_01
file wiki_62
file wiki_10
file wiki_73
file wiki_34
file wiki_25
file wiki_36
file wiki_08
file wiki_07
file wiki_85
file wiki_82
file wiki_45
file wiki_67


# 利用搜索树＋动态规划得到和“说”相近的词

# 使用NER, Dependency Parsing获得是谁说了话，说了什么话

# 句子结束

# 制作网页呈现内容

# 服务器部署